<a href="https://colab.research.google.com/github/payamhooshmand/dadik-AI/blob/main/text_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import gensim
import pandas as pd

# خواندن داده‌های متنی
file_path = "m.csv"
laws_df = pd.read_csv(file_path)

# پردازش متن برای آموزش مدل
sentences = [text.split() for text in laws_df["متن ماده"].dropna()]

# آموزش مدل Word2Vec
word2vec = gensim.models.Word2Vec(sentences, vector_size=100, window=5, min_count=2, workers=4)

# ذخیره مدل برای استفاده بعدی
word2vec.save("word2vec.model")
print("✅ مدل با موفقیت ذخیره شد.")

✅ مدل با موفقیت ذخیره شد.


In [ ]:
import gensim
import pandas as pd

# خواندن داده‌های متنی
file_path = "m.csv"
laws_df = pd.read_csv(file_path)

# پردازش متن برای آموزش مدل
sentences = [text.split() for text in laws_df["متن ماده"].dropna()]

# آموزش مدل Word2Vec
word2vec = gensim.models.Word2Vec(sentences, vector_size=100, window=5, min_count=2, workers=4)

# ذخیره مدل برای استفاده بعدی
word2vec.save("word2vec.model")
print("✅ مدل با موفقیت ذخیره شد.")
import torch
import pandas as pd
import gensim
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# بارگذاری مدل Word2Vec پیش‌آموزش‌دیده (مثلاً مدل فارسی موجود در اینترنت یا مدل آموزش داده‌شده روی متن قوانین)
model_path = "word2vec.model"  # مسیر مدل Word2Vec
word2vec = gensim.models.Word2Vec.load(model_path)

def sentence_vector(sentence, model):
    """تبدیل جمله به بردار با میانگین بردارهای کلمات آن"""
    words = sentence.split()
    word_vectors = [model.wv[word] for word in words if word in model.wv]
    return np.mean(word_vectors, axis=0) if word_vectors else np.zeros(model.vector_size)

# دریافت اطلاعات از کاربر
shakwayeh_ya_eteraz = input("۱. آیا شکواِیِه می‌خواهید یا اعتراض؟ ")
marjae_eteraz = input("۲. به کدام مرجع اعتراض دارید؟ ")
moshkel = input("۳. مشکل خود را بیان کنید: ")

query = f"{shakwayeh_ya_eteraz} {marjae_eteraz} {moshkel}"

# بارگذاری پایگاه داده قوانین
file_path = "m.csv"  # نام فایل دیتاست
laws_df = pd.read_csv(file_path)

# تبدیل قوانین به بردارهای Word2Vec
law_vectors = np.array([sentence_vector(text, word2vec) for text in laws_df["متن ماده"]])

# تبدیل ورودی کاربر به بردار
query_vector = sentence_vector(query, word2vec).reshape(1, -1)

# محاسبه شباهت کسینوسی بین سوال کاربر و قوانین
similarities = cosine_similarity(query_vector, law_vectors).flatten()

# یافتن نزدیک‌ترین قانون
best_match_index = torch.argmax(torch.tensor(similarities)).item()
best_law = laws_df.iloc[best_match_index]

# نمایش قانون مرتبط
law_text = best_law["متن ابلاغیه"]
law_number = best_law["متن ماده"]
print("\n💡 قانون مرتبط با موضوع شما:")
print(f"📜 {law_number}: {law_text}")

# **تولید خودکار متن لایحه یا شکواییه**
if shakwayeh_ya_eteraz.strip() == "شکواییه":
    template = f"""
به نام خدا

ریاست محترم {marjae_eteraz}

موضوع: شکواییه در خصوص {moshkel}

با سلام و احترام،
اینجانب بدین‌وسیله نسبت به {moshkel} معترض بوده و درخواست رسیدگی به این موضوع را دارم.
مطابق با {law_number} قانون که بیان می‌دارد:
«{law_text}»
خواهشمند است دستور رسیدگی و اقدامات لازم را مبذول فرمایید.

با احترام،
نام و نام خانوادگی: …
تاریخ: …
"""
else:
    template = f"""
به نام خدا

ریاست محترم {marjae_eteraz}

موضوع: اعتراض نسبت به {moshkel}

با سلام و احترام،
اینجانب نسبت به {moshkel} اعتراض دارم و درخواست بررسی این موضوع را دارم.
مطابق با {law_number} قانون که بیان می‌دارد:
«{law_text}»
خواهشمند است ضمن بررسی موضوع، اقدامات مقتضی را مبذول فرمایید.

با احترام،
نام و نام خانوادگی: …
تاریخ: …
"""
print(template)

# **ذخیره لایحه در فایل متنی**
output_file = "law_document.txt"
with open(output_file, "w", encoding="utf-8") as file:
    file.write(template)

print("\n✅ لایحه شما با موفقیت ایجاد و ذخیره شد.")
print(f"📂 مسیر فایل: {output_file}")

۱. آیا شکواِیِه می‌خواهید یا اعتراض؟ اعتراض
۲. به کدام مرجع اعتراض دارید؟ دوم اداری 
۳. مشکل خود را بیان کنید: از سه سال پیش مالیات افزوده بر درآمد پرداخت کردم. شغل آزاد دارم و امسال درآمدی نداشتم. دادگاه مرا محکوم به پرداخت مالیات کرده است. اما درآمدی ندارم و خواهان اعتراض هستم.

💡 قانون مرتبط با موضوع شما:
📜 ماده ۹۴ در آمد مشمول مالیات مودیان موضوع این فصل عبارت است از کل فروش کالا و خدمات به اضافه سایر درآمدهای آنان که مشمول مالیات فصول دیگر شناخته نشده پس از کار هزینه ها و استهلاکات مربوط طبق مقررات فصل هزینه های قابل قبول و استهلاکات: شماره ۲۸ ۲۰۷ / ت ۳۳۳۹۱هتاریخ ۰۸ / ۰۵ / ۱۳۸۴ پیوست وزارت کار و امور اجتماعی - وزارت امور اقتصادی و دارایی - سازمان تامین اجتماعی سازمان مدیریت و برنامه ریزی کشور - بانک مرکزی جمهوری اسلامی ایران هیات وزیران در جلسه مورخ ۸ / ۴ / ۱۳۸۴ بنا به پیشنهاد مشترک وزارتخانه های کار و امور اجتماعی تعاون سازمان مدیریت و برنامه ریزی کشور و سازمان تامین اجتماعی و به استناد ماده ( ۴۹ ) تنفیذی قانون برنامه سوم توسعه اقتصادی اجتماعی و فرهنگی جمهوری اسلامی ایران موضوع م

In [ ]:
file_path = "m.csv"  # نام فایل دیتاست
laws_df = pd.read_csv(file_path)
laws_df.describe()

,متن ماده,متن ابلاغیه
count,370,316
unique,370,301
top,ماده ۱ در این قانون اصطلاحات زیر در معانی مشرو...,شماره ۶ ۵۵۸ / ت ۲۳۹۹۵ه تاریخ ۲۲ / ۰۲ / ۱۳۸۱ پی...
freq,1,5


In [21]:

import os
import gensim
import pandas as pd
import numpy as np
import torch
from sklearn.metrics.pairwise import cosine_similarity

def collect_information():
    """جمع‌آوری اطلاعات از کاربر"""
    complaint_type = input("آیا شکواییه می‌خواهید یا اعتراض؟ (شکواییه/اعتراض): ").strip()
    legal_reference = input("مرجع قانونی مورد نظر شما چیست؟ ").strip()

    notification_date = None
    image_path = None
    defendant_name = None

    if complaint_type.lower() == 'اعتراض':
        notification_date = input("تاریخ ابلاغ حکم را وارد کنید (مثال: 1402/12/01): ").strip()
        image_path = input("تصویر حکم را آپلود کنید (مسیر فایل): ").strip()
        if not os.path.exists(image_path):
            print("مسیر فایل صحیح نیست.")
            image_path = None

    branch = input("شعبه مورد نظر را وارد کنید: ").strip()

    if complaint_type.lower() == 'شکواییه':
        defendant_name = input("نام نهاد یا فرد طرف دعوی را وارد کنید: ").strip()

    complaint_subject = input("موضوع شکایت را شرح دهید: ").strip()
    complaint_category = input("موضوع شکایت اداری، سیاسی، خانوادگی یا مالی است؟ ").strip()

    user_data = {
        'complaint_type': complaint_type,
        'legal_reference': legal_reference,
        'notification_date': notification_date,
        'branch': branch,
        'image_path': image_path,
        'defendant_name': defendant_name,
        'complaint_subject': complaint_subject,
        'complaint_category': complaint_category
    }

    return user_data

def load_word2vec_model(model_path="word2vec.model"):
    """بارگذاری مدل Word2Vec"""
    return gensim.models.Word2Vec.load(model_path)

def sentence_vector(sentence, model):
    """تبدیل جمله به بردار با میانگین بردارهای کلمات آن"""
    words = sentence.split()
    word_vectors = [model.wv[word] for word in words if word in model.wv]
    return np.mean(word_vectors, axis=0) if word_vectors else np.zeros(model.vector_size)

def find_best_matching_law(query, model, laws_df):
    """یافتن مرتبط‌ترین قانون با توجه به ورودی کاربر"""
    law_vectors = np.array([sentence_vector(text, model) for text in laws_df["متن ماده"]])
    query_vector = sentence_vector(query, model).reshape(1, -1)

    similarities = cosine_similarity(query_vector, law_vectors).flatten()
    best_match_index = torch.argmax(torch.tensor(similarities)).item()
    best_law = laws_df.iloc[best_match_index]

    return best_law["متن ماده"], best_law["متن ابلاغیه"]

def generate_document(user_data, law_number, law_text):
    """تولید خودکار متن لایحه یا شکواییه"""
    template = f"""
به نام خدا

ریاست محترم {user_data['legal_reference']}

موضوع: {user_data['complaint_type']} در خصوص {user_data['complaint_subject']}

با سلام و احترام،
اینجانب بدین‌وسیله نسبت به {user_data['complaint_subject']} {("معترض" if user_data['complaint_type'] == "اعتراض" else "شاکی")} بوده و درخواست رسیدگی به این موضوع را دارم.
مطابق با {law_number} قانون که بیان می‌دارد:
«{law_text}»
خواهشمند است دستور رسیدگی و اقدامات لازم را مبذول فرمایید.

با احترام،
نام و نام خانوادگی: …
تاریخ: …
"""
    return template

def main():
    """اجرای برنامه"""
    user_data = collect_information()
    query = f"{user_data['complaint_type']} {user_data['legal_reference']} {user_data['complaint_subject']}"

    # بارگذاری مدل و پایگاه داده قوانین
    word2vec = load_word2vec_model()
    laws_df = pd.read_csv("m.csv")

    law_number, law_text = find_best_matching_law(query, word2vec, laws_df)

    # تولید و ذخیره متن
    document_text = generate_document(user_data, law_number, law_text)
    print(document_text)
    output_file = "law_document.txt"
    with open(output_file, "w", encoding="utf-8") as file:
        file.write(document_text)

    print("\n✅ لایحه شما با موفقیت ایجاد و ذخیره شد.")
    print(f"📂 مسیر فایل: {output_file}")

if __name__ == "__main__":
    main()

آیا شکواییه می‌خواهید یا اعتراض؟ (شکواییه/اعتراض): اعتراض 
مرجع قانونی مورد نظر شما چیست؟ عالی
تاریخ ابلاغ حکم را وارد کنید (مثال: 1402/12/01): 
تصویر حکم را آپلود کنید (مسیر فایل): 
مسیر فایل صحیح نیست.
شعبه مورد نظر را وارد کنید: دوم
موضوع شکایت را شرح دهید: از سه سال پیش مالیات پرداخت کردم ولی امسال پولی برای پرداخت مالیات ندارم 
موضوع شکایت اداری، سیاسی، خانوادگی یا مالی است؟ مالی

به نام خدا

ریاست محترم عالی

موضوع: اعتراض در خصوص از سه سال پیش مالیات پرداخت کردم ولی امسال پولی برای پرداخت مالیات ندارم

با سلام و احترام،
اینجانب بدین‌وسیله نسبت به از سه سال پیش مالیات پرداخت کردم ولی امسال پولی برای پرداخت مالیات ندارم معترض بوده و درخواست رسیدگی به این موضوع را دارم.
مطابق با ماده ۱۵۷ نسبت به مودیان مالیات بر درآمد که در موعد مقرر از تسلیم اظهارنامه منبع درآمد خودداری نموده اند یا اصولا طبق مقررات این قانون مکلف به تسلیم اظهارنامه در سر رسید پرداخت مالیات نیستند مرور زمان مالیاتی پنج سال از تاریخ در رسید پرداخت مالیات می باشد و پس از گذشته پنج سال مذکور مالیات متعلق قابل مطالبه 